In [1]:
%load_ext autoreload
%autoreload 2
from preprocess import Preprocessor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

p_train = Preprocessor('train')
p_test = Preprocessor('test')

load course date info over!
number of courses: 39
load course object info over!
number of courses: 39
number of moduels: 26750
load course object info over!
size of log data: 4677908
number of enrollment_ids: 72395
load enrollment info over!
number of courses: 39
number of enrollments: 72395
number of uers:  53870
load truth info over!
number of labels: 72395
load course date info over!
number of courses: 39
load course object info over!
number of courses: 39
number of moduels: 26750
load course object info over!
size of log data: 1548480
number of enrollment_ids: 24013
load enrollment info over!
number of courses: 39
number of enrollments: 24013
number of uers:  21266
load truth info over!
number of labels: 24013


In [2]:
X_train_all, y_train_all = p_train.get_values_all()

event_count features extracted! 1.344419 seconds taken
Shape of the event_count dataframe:  (72395, 8)
weekly_session_count features extracted! 60.504248 seconds taken
Shape of the weekly_session_count dataframe:  (72395, 6)
problem_video_ratio features extracted! 3.183553 seconds taken
Shape of the problem_video dataframe:  (72395, 3)
All features extracted! 65.190826 seconds taken
Shape of the whole features dataframe:  (72395, 16)
Getting train data preprocessing done! 65.286530 seconds taken
Getting raw values for X, y done!
The shape of X: (72395, 15); shape of y: (72395,)


In [3]:
X_test, y_test = p_test.get_values_all()

event_count features extracted! 0.417885 seconds taken
Shape of the event_count dataframe:  (24013, 8)
weekly_session_count features extracted! 18.258213 seconds taken
Shape of the weekly_session_count dataframe:  (24013, 6)
problem_video_ratio features extracted! 1.017272 seconds taken
Shape of the problem_video dataframe:  (24013, 3)
All features extracted! 19.747248 seconds taken
Shape of the whole features dataframe:  (24013, 16)
Getting test data preprocessing done! 19.766196 seconds taken
Getting raw values for X, y done!
The shape of X: (24013, 15); shape of y: (24013,)


# XGBoost

In [112]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=200, max_depth=6, learning_rate=0.05, min_child_weight=2, n_jobs=-1, max_delta_step=1, 
                      objective='binary:logistic', gamma=3 ,subsample=1)
model.fit(X_train_all, y_train_all)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=3, learning_rate=0.05, max_delta_step=1,
       max_depth=6, min_child_weight=2, missing=None, n_estimators=200,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [113]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.77      0.57      0.66      4902
          1       0.90      0.96      0.93     19111

avg / total       0.87      0.88      0.87     24013



C:\Users\Kang\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [114]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.4f%%" % (accuracy * 100.0))

Accuracy: 87.8024%


# Logistic Regression

In [101]:
# Logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

clf = LogisticRegression(tol=1e-3, C=1.5, random_state=0)
clf = clf.fit(X_train_all, y_train_all)
expected = y_test
predicted = clf.predict(X_test)

print("Classification report for classifier %s:\n%s\n"
      % (clf, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))
print ('Score: ')
print ("%.4f%%" % (clf.score(X_test,y_test) * 100.0))
# print ('Cross validation score: ')
# print (cross_val_score(clf, X_train_all, y_train_all, cv=3))

Classification report for classifier LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.001,
          verbose=0, warm_start=False):
             precision    recall  f1-score   support

          0       0.78      0.54      0.64      4902
          1       0.89      0.96      0.92     19111

avg / total       0.87      0.87      0.87     24013


Confusion matrix:
[[ 2624  2278]
 [  736 18375]]
Score: 
87.4485%


# Gradient Boost

In [86]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=200, max_depth=6, learning_rate=0.05, random_state=0)
clf = clf.fit(X_train_all, y_train_all)
expected = y_test
predicted = clf.predict(X_test)

print("Classification report for classifier %s:\n%s\n"
      % (clf, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))
print ('Score: ')
print ("%.4f%%" % (clf.score(X_test,y_test) * 100.0))

Classification report for classifier GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=6,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False):
             precision    recall  f1-score   support

          0       0.77      0.57      0.65      4902
          1       0.90      0.96      0.92     19111

avg / total       0.87      0.88      0.87     24013


Confusion matrix:
[[ 2774  2128]
 [  836 18275]]
Score: 
87.6567%


# Random Forest

In [76]:
clf = RandomForestClassifier(n_estimators=200, max_depth=12, random_state=0, min_samples_split=3, n_jobs=-1)
clf = clf.fit(X_train_all, y_train_all)
expected = y_test
predicted = clf.predict(X_test)

print("Classification report for classifier %s:\n%s\n"
      % (clf, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))
print ('Score: ')
print ("%.4f%%" % (clf.score(X_test,y_test) * 100.0))
# print (cross_val_score(clf, X_train_all, y_train_all, cv=5))

Classification report for classifier RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False):
             precision    recall  f1-score   support

          0       0.77      0.56      0.65      4902
          1       0.89      0.96      0.92     19111

avg / total       0.87      0.88      0.87     24013


Confusion matrix:
[[ 2736  2166]
 [  808 18303]]
Score: 
87.6150%


# MLP

In [84]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(64,32))

clf = clf.fit(X_train_all, y_train_all)
expected = y_test
predicted = clf.predict(X_test)

print("Classification report for classifier %s:\n%s\n"
      % (clf, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))
print ('Score: ')
print ("%.4f%%" % (clf.score(X_test,y_test) * 100.0))
# print (cross_val_score(clf, X_train_all, y_train_all, cv=5))

Classification report for classifier MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(64, 32), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False):
             precision    recall  f1-score   support

          0       0.75      0.56      0.64      4902
          1       0.89      0.95      0.92     19111

avg / total       0.87      0.87      0.87     24013


Confusion matrix:
[[ 2763  2139]
 [  903 18208]]
Score: 
87.3319%


### Feature Importance

In [13]:
f = clf_all.feature_importances_
c = p_train.get_features_all().columns

event_count features extracted! 1.217704 seconds taken
Shape of the event_count dataframe:  (72395, 8)
weekly_session_count features extracted! 51.340733 seconds taken
Shape of the weekly_session_count dataframe:  (72395, 6)
problem_video_ratio features extracted! 3.045905 seconds taken
Shape of the problem_video dataframe:  (72395, 3)
All features extracted! 55.734944 seconds taken
Shape of the whole features dataframe:  (72395, 16)


In [14]:
d={}
for i in range(len(f)):
    d[c[i+1]] = f[i]
d

{'access_count': 0.15930482937723603,
 'discussion_count': 0.04269388956217597,
 'navigate_count': 0.09841600852514756,
 'page_close_count': 0.09697023842232648,
 'problem_count': 0.0692296509961611,
 'video_count': 0.061399968295089785,
 'wiki_count': 0.023401594378286227,
 'week_one_session': 0.023761561767072562,
 'week_two_session': 0.03402435521352029,
 'week_three_session': 0.033797121619515834,
 'week_four_session': 0.05919820710931555,
 'week_five_session': 0.10147390902438054,
 'week_six_session': 0.006813039525642023,
 'problem_ratio': 0.11000736983283753,
 'video_ratio': 0.07950825635129263}

### Partial fit

In [103]:
X_train_balance, y_train_balance = p_train.get_values_partial(0.4)

event_count features extracted! 1.652582 seconds taken
Shape of the event_count dataframe:  (72395, 8)
weekly_session_count features extracted! 54.183138 seconds taken
Shape of the weekly_session_count dataframe:  (72395, 6)
problem_video_ratio features extracted! 3.300178 seconds taken
Shape of the problem_video dataframe:  (72395, 3)
All features extracted! 59.294496 seconds taken
Shape of the whole features dataframe:  (72395, 16)
Getting train data preprocessing done! 59.346373 seconds taken
Getting partial X, y done!
The shape of X: (37975, 15); shape of y: (37975,)
The ratio of 1 in labels:  60.42%


In [104]:
from xgboost import XGBClassifier

model_partial = XGBClassifier(n_estimators=200, max_depth=6, learning_rate=0.05, min_child_weight=2, n_jobs=-1, max_delta_step=1, 
                      objective='binary:logistic', gamma=3 ,subsample=1)
model_partial.fit(X_train_balance, y_train_balance)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=3, learning_rate=0.05, max_delta_step=1,
       max_depth=6, min_child_weight=2, missing=None, n_estimators=200,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [105]:
y_pred = model_partial.predict(X_test)
predictions = [round(value) for value in y_pred]
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.64      0.69      0.67      4902
          1       0.92      0.90      0.91     19111

avg / total       0.86      0.86      0.86     24013



C:\Users\Kang\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [106]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.4f%%" % (accuracy * 100.0))

Accuracy: 85.7619%


In [58]:
clf_partial = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=520)
clf_partial.fit(X_train_balance, y_train_balance)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=520, verbose=0, warm_start=False)

In [59]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf_partial.predict(X_test)))

             precision    recall  f1-score   support

          0       0.59      0.71      0.64      4902
          1       0.92      0.87      0.90     19111

avg / total       0.85      0.84      0.84     24013

